In [8]:
import sys
sys.path.append("..")
import json
from tqdm import tqdm

from utils.data import get_data_elements

In [5]:
with open("./../../datasets/finetuning_dataset/val.json", "r") as f:
    data = json.load(f)

In [7]:
def get_img_path(ann):
    visual_news_data = json.load(open("../../datasets/visualnews/origin/data.json"))
    visual_news_data_mapping = {ann["id"]: ann for ann in visual_news_data}


    caption = visual_news_data_mapping[ann["id"]]["caption"]
    image_path = visual_news_data_mapping[ann["image_id"]]["image_path"]
    image_path = "./visualnews/origin/"+image_path[2:]
    return image_path

In [9]:
ready_dataset = []
for dp in tqdm(data):
    true_ann, false_ann = dp['true_ann'], dp['false_ann']
    data_dict = {}
    _, true_cap = get_data_elements(true_ann)
    _, false_cap = get_data_elements(false_ann)
    data_dict["id"] = str(true_ann["id"])+"_"+str(false_ann['id'])
    data_dict["image"] = get_img_path(dp['true_ann'])
    data_dict["conversations"] = [
      {
        "from": "human",
        "value": """<image>\nYou are given two captions for the given image. One caption is True and the other is False.
                Based on the image, your task is to identify the most obvious element of inconsistency (MUST BE ONE WORD) such as location, time, event, person, organisation etc. in the two captions.
                TRUE CAPTION: {}
                FALSE CAPTION: {}

                You must respond by filling the following template as accurately as possible. 
                Be as concise as possble.
                <template>
                The two captions are inconsistent in: <element>. 
                The <element> in the false caption is: <false_entity>.
                However, the <element> which the true caption and image correspond to is: <true_entity>
                </template>""".format(true_cap, false_cap)
      },
      {
        "from": "gpt",
        "value": """The two captions are inconsistent in: {}. 
                The {} in the false caption is: {}.
                However, the {} which the true caption and image correspond to is: {}""".format(dp['inconsistent_entity'], dp['inconsistent_entity'],dp['false_entity'],dp['inconsistent_entity'], dp['true_entity'])
      },
    ]
    ready_dataset.append(data_dict)

  0%|          | 11/5356 [00:53<7:13:06,  4.86s/it]

In [6]:
ready_dataset[0]

{'id': '92318_136816',
 'image': '../../datasets/visualnews/origin/washington_post/images/0218/348.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nYou are given two captions for the given image. One caption is True and the other is False.\n                Based on the image, your task is to identify the most obvious element of inconsistency (MUST BE ONE WORD) such as location, time, event, person, organisation etc. in the two captions.\n                TRUE CAPTION: People walk in the parking lot of an Ikea store in Brooklyn\n                FALSE CAPTION: An an armed suspect shot an armored car guard during a robbery in the Oxon Hill area of Prince George s County authorities said\n\n                You must respond by filling the following template as accurately as possible. \n                Be as concise as possble.\n                <template>\n                The two captions are inconsistent in: <element>. \n                The <element> in the false caption is: 

In [8]:
with open("./../../datasets/finetuning_dataset/val.json", "w") as f:
    json.dump(ready_dataset, f)